In [2]:
# Imports
import os

import numpy as np

import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

import rclpy
from rclpy.node import Node

from sensor_msgs.msg import Image
from audio_common_msgs.msg import AudioDataStamped
from situated_hri_interfaces.srv import SceneVisRec, SceneAudioRec

from std_srvs.srv import Empty

In [3]:
# Parameters
filepath = '../../bags/e1_est_tuning'

In [4]:
# Member variables
rclpy.init()
node = Node('analysis_script_node')

actual_scene_labels = ['campus','courtyard','lab','lobby']

audio_data_pub = node.create_publisher(AudioDataStamped, 'audio_data',10)
img_data_pub = node.create_publisher(Image, 'oak/rgb/image_raw',10)

reset_bayes_audio_scene_client = node.create_client(Empty, 'bayes_audio_scene_est/reset') 
reset_clip_scene_client = node.create_client(Empty, 'clip_scene_rec/reset')

# Helper functions
def typename(topic_name):
    for topic_type in topic_types:
        if topic_type.name == topic_name:
            return topic_type.type
    raise ValueError(f"topic {topic_name} not in bag")

In [ ]:
# Traverse filepath
for root, dirs, files in os.walk(filepath):

    # Examine each file
    for file in files:
        
        # If it is an mcap, examine
        if os.path.splitext(file)[-1] == '.mcap':

            # Get labels based on filepath
            labels = os.path.normpath(root).split(os.sep)
            command_actual = labels[-1]
            modality_actual = labels[-2]
            role_actual = labels[-3]
            scene_actual = labels[-4]

            scene_actual_idx = actual_scene_labels.index(scene_actual)
            
            print('Command: %s\nModality: %s\nRole: %s\nScene: %s (%s)\n\n' % (command_actual, modality_actual, role_actual, scene_actual, scene_actual_idx))
            
            reader = rosbag2_py.SequentialReader()            
            reader.open(
                rosbag2_py.StorageOptions(uri=os.path.join(root,file), storage_id="mcap"),
                rosbag2_py.ConverterOptions(
                    input_serialization_format="cdr", output_serialization_format="cdr"
                ),
            )
            topic_types = reader.get_all_topics_and_types()

            # Iterate through messages
            while reader.has_next():
                topic, data, timestamp = reader.read_next()
                msg_type = get_message(typename(topic))

                # Handle audio scene data
                if topic=='/audio_data':
                    audio_data_pub.publish(deserialize_message(data,msg_type))

                if topic=='/oak/rgb/image_raw':
                    img_data_pub.publish(deserialize_message(data,msg_type))
                
            del reader

            # Reset estimator nodes
            print('Resetting')
            
            bayes_audio_future = reset_bayes_audio_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, bayes_audio_future)
            
            clip_future = reset_clip_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, clip_future)

# Cleanup
rclpy.shutdown()

Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: follow-me
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Resetting
Command: follow-me
Modality: gest_verb
Rol

In [1]:
rclpy.shutdown()

NameError: name 'rclpy' is not defined

In [ ]:
# For file in filepath, get scene, role, modality, and command

# Reset estimator nodes

# LAST - Load new parameters

# Play bag file

# Record scene responses - audio, visual, fused

# Record command responses - ARtag, speech, gesture, fused